In [23]:
import datajoint as dj
import numpy as np
import pathlib
import re

import pynwb
from pynwb import NWBHDF5IO

from pipeline import experiment, intracellular, subject
from pipeline.nwb_adapter import nwb_file

## Define a generic "fetch_nwb" method that returns nwb object given object_id

In [27]:
def fetch_nwb(relation, nwb_orig, *attrs, **kwargs):
    """
    :param relation: a DJ relation to call fetch on
    :param nwb_orig: tuple of (table, attr) to get
    :param attrs: attrs from normal fetch()
    :param kwargs: kwargs from normal fetch()
    :return: fetched list of dict
    """
    kwargs['as_dict'] = True  # force return as dictionary
    tbl, attr_name = nwb_orig
    
    if not attrs:
        attrs = relation.heading.names

    rec_dicts = (relation * tbl.proj(nwb2load_filepath=attr_name)).fetch(*attrs, 'nwb2load_filepath', **kwargs)
    
    if not rec_dicts or not np.any(['object_id' in key for key in rec_dicts[0]]):
        return rec_dicts
    
    ret = []
    for rec_dict in rec_dicts:
        io = pynwb.NWBHDF5IO(rec_dict.pop('nwb2load_filepath'), mode='r')
        nwbf = io.read()
        nwb_objs = {re.sub('(_?)object_id', '', id_attr): nwbf.objects[rec_dict[id_attr]]
                    for id_attr in attrs if 'object_id' in id_attr}
        ret.append({**rec_dict, **nwb_objs})
    return ret

## Usage 1 - Refering to filepath in `self`

In [47]:
schema = experiment.schema

@schema
class NWBSession(dj.Manual):
    definition = """
     -> subject.Subject
    session_time         : datetime                     # session time
    ---
    nwb_fp: filepath@nwb_store
    nwb_object_id: varchar(36)
    """
    
    def load(self):
        for key in (experiment.Session - self).fetch('KEY'):
            nwb = (experiment.Session & key).fetch1('nwb_file')
            print(nwb.container_source)
            self.insert1({**key, 
                          'nwb_fp': pathlib.Path(nwb.container_source),
                          'nwb_object_id': nwb.object_id})
            
    def fetch_nwb(self, *attrs, **kwargs):
        return fetch_nwb(self, (self, 'nwb_fp'), *attrs, **kwargs)

In [48]:
NWBSession().load()

In [49]:
NWBSession().fetch_nwb(limit=1)

[{'subject_id': 'anm300476',
  'session_time': datetime.datetime(2017, 5, 20, 18, 28, 59),
  'nwb_fp': 'C:\\Users\\thinh\\Documents\\TN-Vathes\\NWB\\nwb_hackathons_2020\\nwb_store\\session\\anm300476_170520_182859.nwb',
  'nwb_object_id': '2aa21234-7536-4125-a2aa-da0161fa7f6b',
  'nwb': 
  root <class 'pynwb.file.NWBFile'>
  Fields:
    experimenter: ['Hidehiko Inagaki']
    file_create_date: [datetime.datetime(2020, 5, 11, 13, 50, 49, 394621, tzinfo=tzoffset(None, -18000))]
    identifier: anm300476_170520_182859
    session_description: ALM whole cell recording during M1 photoinhibition
    session_start_time: 2017-05-20 18:28:59-05:00
    subject: subject <class 'pynwb.file.Subject'>
    timestamps_reference_time: 2017-05-20 18:28:59-05:00}]

Value of the `nwb`-key is fetched back as an NWB object with the corresponding `object_id` - In the case the whole NWBFile

## Usage 2 - Refering to filepath from upstream parent table

In [54]:
schema = experiment.schema

@schema
class NWBSubject(dj.Imported):
    definition = """
     -> NWBSession
    ---
    subject_object_id: varchar(36)
    """
    
    def make(self, key):
        io = pynwb.NWBHDF5IO((NWBSession & key).fetch1('nwb_fp'), mode='r')
        nwbf = io.read()
        self.insert1({**key, 'subject_object_id': nwbf.subject.object_id})
            
    def fetch_nwb(self, *attrs, **kwargs):
        return fetch_nwb(self, (NWBSession, 'nwb_fp'), *attrs, **kwargs)

In [55]:
NWBSubject.populate()

In [57]:
NWBSubject().fetch_nwb(limit=1)

[{'subject_id': 'anm300476',
  'session_time': datetime.datetime(2017, 5, 20, 18, 28, 59),
  'subject_object_id': '00e9e60c-583b-4729-901a-2b27bd19e974',
  'subject': 
  subject <class 'pynwb.file.Subject'>
  Fields:
    date_of_birth: 2015-03-28 00:00:00-05:00
    genotype: PV-Cre x Ai32
    sex: M
    species: Mus musculus
    subject_id: anm300476}]

Value of the `nwb`-key is fetched back as an NWB object with the corresponding `object_id` - In the case the `.subject`